https://github.com/explosion/spaCy/blob/master/LICENSE

Example of training spaCy's named entity recognizer, starting off with an
existing model or a blank model.

For more details, see the documentation:
* Training: https://spacy.io/usage/training#section-ner
* NER: https://spacy.io/usage/linguistic-features#named-entities

In [1]:
import random
from pathlib import Path
import spacy

In [2]:
TRAIN_DATA = [
    ('Who is Shaka Khan?', {
        'entities': [(7, 17, 'PERSON')]
    }),
    ('I like London and Berlin.', {
        'entities': [(7, 13, 'LOC'), (18, 24, 'LOC')]
    })
]

In [3]:
nlp = spacy.blank('en')

In [4]:
ner = nlp.create_pipe('ner')
nlp.add_pipe(ner, last=True)

In [5]:
# add labels
for _, annotations in TRAIN_DATA:
    for ent in annotations.get('entities'):
        ner.add_label(ent[2])

In [6]:
# get names of other pipes to disable them during training
other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
with nlp.disable_pipes(*other_pipes):  # only train NER
    optimizer = nlp.begin_training()
    for itn in range(100):
        random.shuffle(TRAIN_DATA)
        losses = {}
        for text, annotations in TRAIN_DATA:
            nlp.update(
                [text],  # batch of texts
                [annotations],  # batch of annotations
                drop=0.5,  # dropout - make it harder to memorise data
                sgd=optimizer,  # callable to update weights
                losses=losses)
        print(losses)

{'ner': 16.235757490620017}
{'ner': 13.01175731967129}
{'ner': 10.872870404702994}
{'ner': 6.7520108227067865}
{'ner': 11.270936255721608}
{'ner': 8.677501894709211}
{'ner': 6.5614776042387035}
{'ner': 7.317424037225578}
{'ner': 9.39236497294058}
{'ner': 5.926565954928012}
{'ner': 5.80514592810008}
{'ner': 9.848136051044275}
{'ner': 5.21263195669442}
{'ner': 2.311033142309743}
{'ner': 3.961020756766131}
{'ner': 5.114914817656618}
{'ner': 0.01482071524596984}
{'ner': 2.0738199233090326}
{'ner': 0.8535795233154776}
{'ner': 0.10320643632971646}
{'ner': 3.2304784020091533}
{'ner': 3.9366866879333484}
{'ner': 0.45400242268461977}
{'ner': 2.4191047049540034e-06}
{'ner': 1.3981112792385961}
{'ner': 0.008570982444339582}
{'ner': 0.00023454407742679525}
{'ner': 1.1047200503546818e-05}
{'ner': 9.507984257665279e-06}
{'ner': 1.1411061163155201e-07}
{'ner': 0.00023665812537962782}
{'ner': 1.8445421464827263}
{'ner': 4.849918925771717e-07}
{'ner': 4.654769725250924e-08}
{'ner': 0.000296644540269873

In [7]:
# test the trained model
for text, _ in TRAIN_DATA:
    doc = nlp(text)
    print('Entities', [(ent.text, ent.label_) for ent in doc.ents])
    print('Tokens', [(t.text, t.ent_type_, t.ent_iob) for t in doc])

Entities [('Shaka Khan', 'PERSON')]
Tokens [('Who', '', 2), ('is', '', 2), ('Shaka', 'PERSON', 3), ('Khan', 'PERSON', 1), ('?', '', 2)]
Entities [('London', 'LOC'), ('Berlin', 'LOC')]
Tokens [('I', '', 2), ('like', '', 2), ('London', 'LOC', 3), ('and', '', 2), ('Berlin', 'LOC', 3), ('.', '', 2)]
